In [140]:
import vimba
from vimba import *
import sys
from typing import Optional
import cv2
from time import sleep
import numpy as np

In [14]:
with Vimba.get_instance() as vimba:
    cams = vimba.get_all_cameras()

    print('Cameras found: {}'.format(len(cams)))

    for cam in cams:
        cameraID = cam.get_id()
print(cameraID)

Cameras found: 1
DEV_1AB22C00CD6F


In [55]:
def get_camera(camera_id: Optional[str]) -> Camera:
    with Vimba.get_instance() as vimba:
        if camera_id:
            try:
                return vimba.get_camera_by_id(camera_id)

            except VimbaCameraError:
                print('Failed to access Camera \'{}\'. Abort.'.format(camera_id))

        else:
            cams = vimba.get_all_cameras()
            if not cams:
                print('No Cameras accessible. Abort.')

            return cams[0]

In [56]:
cam = get_camera(cameraID)

In [57]:
cam

In [59]:
settings_file = r'C:\Users\BMLab21\Desktop\camera\initial_settings.xml'

with Vimba.get_instance():
    with get_camera(cameraID) as cam:
        cam.UserSetSelector.set('Default')
        cam.load_settings(settings_file, PersistType.All)
        print("--> Feature values have been loaded from given file '%s'" % settings_file)

--> Feature values have been loaded from given file 'C:\Users\BMLab21\Desktop\camera\initial_settings.xml'


In [62]:
def frame_handler(cam: Camera, frame: Frame):
    if frame.get_status() == FrameStatus.Complete:
        print('Frame(ID: {}) has been received.'.format(frame.get_id()), flush=True)

    cam.queue_frame(frame)

In [67]:
with Vimba.get_instance():
    cam = get_camera(cameraID)
    sender = cam.get_interface_id()

    with cam:
        cam.start_streaming(frame_handler)

Frame(ID: 0) has been received.


In [160]:
def frame_handler2(cam: Camera, frame: Frame):
    c = cam.queue_frame(frame)
    s = frame.convert_pixel_format
    s = frame.as_opencv_image()
    

In [161]:
with Vimba.get_instance() as vimba:
    with get_camera(cameraID) as camera:
        camera.UserSetSelector.set('Default')
        camera.load_settings(settings_file, PersistType.All)
        print("--> Feature values have been loaded from given file '%s'" % settings_file)

        # arm the camera and provide a function to be called upon frame ready
        with camera:
            frames = camera.start_streaming(frame_handler2)

            # stream images for a while...
            sleep(20)

            # stop frame acquisition
            # start_frame_acquisition can simply be called again if the camera is still armed
            camera.stop_streaming()

cv2.destroyAllWindows()

--> Feature values have been loaded from given file 'C:\Users\BMLab21\Desktop\camera\initial_settings.xml'


TypeError: missing a required argument: 'target_fmt'

TypeError: missing a required argument: 'target_fmt'

TypeError: missing a required argument: 'target_fmt'

TypeError: missing a required argument: 'target_fmt'

...

TypeError: missing a required argument: 'target_fmt'

TypeError: missing a required argument: 'target_fmt'

TypeError: missing a required argument: 'target_fmt'

TypeError: missing a required argument: 'target_fmt'

TypeError: missing a required argument: 'target_fmt'